In [1]:
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain.retrievers import ParentDocumentRetriever

In [2]:
## Text Splitting & Docloader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryStore


## BGE Embeddings

In [5]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
device = "cpu"
bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': device},
    encode_kwargs=encode_kwargs
)

c:\Users\shadabamd\AppData\Local\anaconda3\envs\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
.gitattributes: 100%|██████████| 1.52k/1.52k [00:00<?, ?B/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<?, ?B/s] 
README.md: 100%|██████████| 90.3k/90.3k [00:00<00:00, 302kB/s]
config_sentence_transformers.json: 100%|██████████| 124/124 [00:00<00:00, 20.2kB/s]
model.safetensors: 100%|██████████| 133M/133M [00:10<00:00, 12.9MB/s] 
pytorch_model.bin: 100%|██████████| 134M/134M [00:10<00:00, 12.6MB/s] 
sentence_bert_config.json: 100%|██████████| 52.0/52.0 [00:00<00:00, 5.83kB/s]
special_tokens_map.json: 100%|██████████| 125/125 [00:00<00:00, 29.9kB/s]
tokenizer.json: 100%|██████████| 711k/711k [00:00<00:00, 1.49MB/s]
tokenizer_config.json: 100%|██████████| 366/366 [00:00<?, ?B/s] 
vocab.txt: 100%|████

In [9]:
print(len(bge_embeddings.embed_query("hello world, iron man")))

384


### Data prep

In [ ]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
loader = DirectoryLoader('../', glob="**/*.md", loader_cls=TextLoader)